## <font color='black'>Purpose: 
Wouldn't it be amazing if you could be certain your marriage will be successful?
Let us help you figure out if this is the right person for you!</font>

### <font color='gray'>What You Need to Do:
    Simply answer a few quick questions and we will predict if you're destined for marital bliss.

In [341]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import collections
from IPython.display import Image
from sklearn.externals.six import StringIO
from sklearn.linear_model import LogisticRegression

import seaborn as sns
from IPython.display import Image 

In [342]:
fileName = 'marriage.csv'

In [343]:
marriage_raw = pd.read_csv(fileName, header =0)

In [344]:
#select the columns we want
marriage1 = marriage_raw[['m2','state','usr1','spouse','qs1','marital','m4','lwp3','e3','q1','q15a','q15b','q15c','q15d','q15e','q15f','q15g','q15h','q15i','q16','e5','fam2','ownrent','relig','income','ideo','receduc','racethn']]

In [345]:
marriage1.head()

,m2,state,usr1,spouse,qs1,marital,m4,lwp3,e3,q1,...,q15i,q16,e5,fam2,ownrent,relig,income,ideo,receduc,racethn
0,,36,2,2,1,1,,,1,2,...,2,2,1,1,1,9,4,4,4,1
1,,24,2,1,,1,,1,1,2,...,2,2,3,1,1,1,7,3,3,1
2,,44,2,1,,1,,,9,2,...,1,2,9,1,1,1,4,4,2,1
3,,39,1,,,3,,,1,2,...,2,2,,1,1,1,5,3,4,1
4,2,12,2,,,2,,,2,2,...,1,2,,1,1,8,7,5,4,1


In [346]:
#rename columns so we can understand
marriage1 = marriage1.rename(columns={'m2':'everMarried','state': 'state', 'usr1': 'community', 'spouse': 'whichSpouse', 'qs1': 'age', 'marital': 'marriageStatus', 'm4': 'everDivorced', 'lwp3': 'livedTogether', 'e3': 'employment', 'q1': 'lifeSatisfaction','q15a':'faithfulness','q15b':'adeqIncome', 'q15c':'shareRelig', 'q15d':'goodHouse', 'q15e':'agreePolitics', 'q15f':'goodSexLife', 'q15g':'shareChores', 'q15h':'children', 'q15i':'commonInterests', 'q16':'divorceViews', 'e5':'spouseEmployment', 'fam2':'parentMaritalStatus', 'ownrent':'ownrent', 'relig':'religion', 'income':'incomepastyr', 'ideo':'politicalViews', 'receduc':'educationLevel', 'raceeth':'racethn'})

In [347]:
#move everDivorced to end since it is the target
features = list(marriage1.columns.values)
features.pop(features.index('marriageStatus'))
features.pop(features.index('everDivorced'))
marriage1 = marriage1[['marriageStatus']+features+['everDivorced']]

In [348]:
#not sure if we need to convert all columns to integers
marriage1 = marriage1.replace(' ', 0)
marriage1 = marriage1.apply(pd.to_numeric, errors='ignore')
marriage1.dtypes

marriageStatus         int64
everMarried            int64
state                  int64
community              int64
whichSpouse            int64
age                    int64
livedTogether          int64
employment             int64
lifeSatisfaction       int64
faithfulness           int64
adeqIncome             int64
shareRelig             int64
goodHouse              int64
agreePolitics          int64
goodSexLife            int64
shareChores            int64
children               int64
commonInterests        int64
divorceViews           int64
spouseEmployment       int64
parentMaritalStatus    int64
ownrent                int64
religion               int64
incomepastyr           int64
politicalViews         int64
educationLevel         int64
racethn                int64
everDivorced           int64
dtype: object

In [349]:
#remove all rows where MarriageStatus = 6 - never been married or 9 - don't know/refused
marriage1 = marriage1.drop(marriage1[(marriage1.marriageStatus == 6) | (marriage1.marriageStatus == 9)].index)

In [350]:
#check all columns removed
marriage1.loc[marriage1['marriageStatus'].isin([6,9])]

,marriageStatus,everMarried,state,community,whichSpouse,age,livedTogether,employment,lifeSatisfaction,faithfulness,...,divorceViews,spouseEmployment,parentMaritalStatus,ownrent,religion,incomepastyr,politicalViews,educationLevel,racethn,everDivorced


In [351]:
#for all rows where marriage status is 3 - divorced or 4 - separated enter 1 in everDivorced
marriage1.loc[marriage1['marriageStatus'] == 3, 'everDivorced'] = 1
marriage1.loc[marriage1['marriageStatus'] == 4, 'everDivorced'] = 1

In [352]:
#for all rows where everMarried = 1 and marriage status = 2 enter 1 in everDivorced
marriage1.loc[(marriage1['everMarried']==1) & (marriage1['marriageStatus']==2),'everDivorced'] = 1

In [353]:
#for all rows where marriage status is 1 - married or 2 - living together or 5 - widowed enter 2 in everDivorced
marriage1.loc[(marriage1['marriageStatus'].isin([1,2,5])) & (marriage1['everDivorced']==0),'everDivorced'] = 0

In [354]:
marriage1.loc[marriage1['everDivorced'] == 2, 'everDivorced'] = 0

In [355]:
#check 0s and 1s
marriage1

,marriageStatus,everMarried,state,community,whichSpouse,age,livedTogether,employment,lifeSatisfaction,faithfulness,...,divorceViews,spouseEmployment,parentMaritalStatus,ownrent,religion,incomepastyr,politicalViews,educationLevel,racethn,everDivorced
0,1,0,36,2,2,1,0,1,2,1,...,2,1,1,1,9,4,4,4,1,0
1,1,0,24,2,1,0,1,1,2,2,...,2,3,1,1,1,7,3,3,1,0
2,1,0,44,2,1,0,0,9,2,2,...,2,9,1,1,1,4,4,2,1,0
3,3,0,39,1,0,0,0,1,2,1,...,2,0,1,1,1,5,3,4,1,1
4,2,2,12,2,0,0,0,2,2,1,...,2,0,1,1,8,7,5,4,1,0
5,5,0,45,2,0,0,0,3,1,9,...,2,0,1,1,1,10,9,1,2,0
6,1,0,51,3,1,0,9,3,1,1,...,1,2,1,1,8,6,3,4,1,1
7,1,0,47,1,2,0,0,1,1,1,...,2,2,1,1,1,6,2,4,1,0
8,3,0,18,3,0,0,0,1,1,1,...,1,0,1,1,1,6,2,4,1,1
9,5,0,47,1,0,0,0,3,2,1,...,2,0,1,1,1,1,4,2,1,0


In [356]:
df3=marriage1

In [357]:
df2=pd.get_dummies(df3,columns=['state','community',
                        'whichSpouse','employment','lifeSatisfaction','faithfulness','adeqIncome','shareRelig','goodHouse','agreePolitics','goodSexLife','shareChores','children','commonInterests','divorceViews','spouseEmployment','parentMaritalStatus','ownrent','religion','incomepastyr','politicalViews','educationLevel','racethn' ],               
                   dummy_na=True)

In [358]:
X=df2.drop(['everMarried','marriageStatus','everDivorced'],axis=1)

In [359]:
Y=df2.everDivorced

In [360]:
X.isnull().any()

age                   False
livedTogether         False
state_1.0             False
state_4.0             False
state_5.0             False
state_6.0             False
state_8.0             False
state_9.0             False
state_10.0            False
state_11.0            False
state_12.0            False
state_13.0            False
state_16.0            False
state_17.0            False
state_18.0            False
state_19.0            False
state_20.0            False
state_21.0            False
state_22.0            False
state_23.0            False
state_24.0            False
state_25.0            False
state_26.0            False
state_27.0            False
state_28.0            False
state_29.0            False
state_30.0            False
state_31.0            False
state_32.0            False
state_33.0            False
                      ...  
incomepastyr_1.0      False
incomepastyr_2.0      False
incomepastyr_3.0      False
incomepastyr_4.0      False
incomepastyr_5.0    

In [361]:
#break the data up into training and testing sets
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [362]:
marriage1.corr()

,marriageStatus,everMarried,state,community,whichSpouse,age,livedTogether,employment,lifeSatisfaction,faithfulness,...,divorceViews,spouseEmployment,parentMaritalStatus,ownrent,religion,incomepastyr,politicalViews,educationLevel,racethn,everDivorced
marriageStatus,1.000000,0.006933,0.043385,-0.025354,-0.773352,-0.121429,-0.237629,0.272229,0.100674,0.034814,...,0.127091,-0.621105,-0.011647,0.159282,-0.074941,-0.243785,0.059114,-0.098735,-0.035822,0.252940
everMarried,0.006933,1.000000,-0.012556,-0.000054,-0.287967,0.053631,-0.088484,-0.043629,0.020695,-0.010282,...,-0.001156,-0.231276,0.100597,0.099960,0.093128,-0.054942,0.059491,-0.043084,0.065064,-0.082337
state,0.043385,-0.012556,1.000000,0.097486,-0.013667,0.018180,-0.036759,0.001004,0.024285,-0.027215,...,-0.011369,-0.033869,0.002998,-0.031328,-0.043148,-0.002380,-0.010816,-0.002437,-0.059823,-0.000107
community,-0.025354,-0.000054,0.097486,1.000000,0.041511,-0.045814,0.009804,0.012864,-0.027068,-0.012283,...,-0.019673,0.040130,-0.045102,-0.093540,-0.046222,-0.014728,-0.000175,-0.001501,-0.167887,0.002828
whichSpouse,-0.773352,-0.287967,-0.013667,0.041511,1.000000,0.087041,0.217216,-0.088683,-0.124985,-0.042789,...,-0.074711,0.537578,-0.008183,-0.190997,-0.020477,0.229245,-0.046609,0.056682,0.007686,-0.347533
age,-0.121429,0.053631,0.018180,-0.045814,0.087041,1.000000,0.067656,-0.147477,-0.019041,-0.014119,...,-0.035743,-0.005851,0.003467,0.028007,0.060980,-0.002753,0.010145,0.012985,-0.004977,-0.049439
livedTogether,-0.237629,-0.088484,-0.036759,0.009804,0.217216,0.067656,1.000000,-0.129911,0.031933,-0.029912,...,-0.055762,0.129520,0.026504,-0.019012,0.095221,0.035075,0.049943,0.033610,0.020706,0.006271
employment,0.272229,-0.043629,0.001004,0.012864,-0.088683,-0.147477,-0.129911,1.000000,0.059892,0.077477,...,0.133507,0.009501,-0.089331,0.078815,-0.033040,-0.150880,0.101197,-0.182617,-0.013098,0.011775
lifeSatisfaction,0.100674,0.020695,0.024285,-0.027068,-0.124985,-0.019041,0.031933,0.059892,1.000000,0.120302,...,0.120315,-0.090650,0.038033,0.211447,0.023190,-0.047275,0.114354,-0.111196,0.057346,0.123767
faithfulness,0.034814,-0.010282,-0.027215,-0.012283,-0.042789,-0.014119,-0.029912,0.077477,0.120302,1.000000,...,0.202627,0.051538,-0.000479,0.123710,0.022526,-0.028998,0.107203,-0.042850,0.050990,-0.012598


In [363]:
clf = LogisticRegression()
clf.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [364]:
Y=np.ravel(Y)

In [365]:
print ("score Scikit learn: ", clf.score(X_test,Y_test))

('score Scikit learn: ', 0.72916666666666663)


In [366]:
clf.score(X_test,Y_test)

0.72916666666666663

In [367]:
model = LogisticRegression()

In [368]:
model= model.fit(X,Y)

In [369]:
model.score(X,Y)

0.74466750313676289

In [370]:
y_pred = clf.predict(X_test)

In [371]:
clf.predict(X_test)

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 0, 0], dtype=int64)

In [372]:
y_pred[:20]

array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], dtype=int64)

In [373]:
y_pred_proba = clf.predict_proba(X_test)[:,1]

In [374]:
y_pred_proba[:20]

array([ 0.09895991,  0.76377327,  0.61701438,  0.04857375,  0.39288301,
        0.34718476,  0.3769318 ,  0.3718006 ,  0.1791863 ,  0.1984482 ,
        0.45698249,  0.06043744,  0.26915701,  0.36734314,  0.20350749,
        0.30858389,  0.80145375,  0.66627473,  0.80750411,  0.76814238])

In [375]:
(y_pred == Y_test).mean()

0.72916666666666663